На страницах http://mediametrics.ru/data/archive/ размещены ежедневные архивы со "слепками" рейтинга новостей в течение дня.

На первом уровне идет разделение по тому, показатели за какой период находятся в слепках:

day - посещаемость за 24 часа;

hour - посещаемость за 1 час;

online - посещаемость за 10 минут.

Слепки с разными показателями делаются с разной частотой: с суточными показателями - раз в 5 минут, с часовыми - раз в минуту, с 10-минутными - раз в 10 секунд, поэтому самые большие архивы в каталоге online, в каждом архиве 8640 файлов-слепков.

Архивы создаются в течение нескольких минут после полуночи по московскому времени.

Внутри каталогов размещены ежедневные архивы по трем странам: России, Украине и Беларуси.

Например, рассмотрим архив http://mediametrics.ru/data/archive/day/ru-2014-04-01.zip
внутри него списки новостей с суточными показателями (day), по России (ru) за 1 апреля (2014-04-01).

В данном архиве 288 файлов (раз в 5 минут за день):
 day/ru-2014-04-01_00:00:00.tsv
 day/ru-2014-04-01_00:05:00.tsv
 day/ru-2014-04-01_00:10:00.tsv
и так далее. Каждый файл содержит список новостей на тот момент, который обозначен в имени файла (время московское), в списки записывается не более 500 новостей.

TSV - это текстовый формат данных, в котором колонки данных разделены символом табуляции.

В первой строчке находится обозначение колонок и служебные данные:

**URL  Title  Visitors  Delta  Shift  ID  68201  1396296000**

- URL - адрес страницы новости;
- Title - заголовок новости;
- Visitors - количество посетителей за выбранный период;
- Delta - разница в количестве посетителей с момента предыдущей генерации рейтинга (это частота отличается от частоты слепков за час и сутки, часовой и суточный рейтинг создаются раз в 10 и 30 секунд)
- Shift - условный показатель движения новости по позициям рейтинга вверх (меньше нуля) или вниз (больше нуля);
- ID - внутренний идентификатор новости, целое число (один идентификатор может быть присвоен разным новостям, появившихся с разницей более 3 месяцев);
- 68201 - общее количество новостей в списке, но в слепок попадают только первые 500;
- 1396296000 - время обновления рейтинга в виде так называемого unix timestamp (количество секунд с 1 января 1970 года)

In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
import pandas as pd 
import matplotlib #collection of functions for scientific and publication-ready visualization
import numpy as np #foundational package for scientific computing
import scipy as sp #collection of functions for scientific computing and advance mathematics
import os, os.path
import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
import re
import warnings
warnings.filterwarnings('ignore')

import pickle
import zipfile
from tqdm import tqdm

from urllib.request import urlopen
from bs4 import BeautifulSoup


import jupyter_loader
import Loader
import theme_loader # загружаем загрузчик тем для поиска

importing Jupyter notebook from Loader.ipynb
Wall time: 0 ns


In [3]:
period = input('Period - day or hour or month or online or week : ') # выбираем период по умолчанию week
if period == '':
        period = 'week'
url = 'http://mediametrics.ru/data/archive/' +  period # загружаем страницу с выбранным перечнем архивов
html = urlopen(url)
bsObj = BeautifulSoup(html.read(), "lxml")
titles = bsObj.find_all('pre')
m = str(titles).split('</a>')
d = [m[i][-14:-4] for i in range(len(m)) if m[i].endswith('.zip')][:-1]

Period - day or hour or month or online or week : 


In [4]:
# вводим период времени для загрузки
print('Enter start and end date of period in YYYY-MM-DD format')
print('Or use default start and end date -', d[0], d[-1])
date_start = input('Enter a start date in YYYY-MM-DD format :')
date_end = input('Enter a end date in YYYY-MM-DD format :')

Enter start and end date of period in YYYY-MM-DD format
Or use default start and end date - 2018-10-03 2019-01-20
Enter a start date in YYYY-MM-DD format :2018-10-03
Enter a end date in YYYY-MM-DD format :2018-10-05


In [5]:
if not(date_start or date_end):
    date_start = d[0]
    date_end = d[-1]

date = Loader.Loader(date_start, date_end, period)
date.make_url()

100%|██████████| 2/2 [01:12<00:00, 24.28s/it]


In [6]:
import pandasmaker
#from imp import reload
#reload(pandasmaker)

pandas_file = pandasmaker.pandasmaker('./' + date.period)
pandas_file.make_pandas()

importing Jupyter notebook from pandasmaker.ipynb


100%|██████████| 168/168 [00:16<00:00,  6.05it/s]


In [8]:
# сохраняем dataframe в файл
names = pickle.load(open([name for name in os.listdir('./') if name.endswith('.pic')][0], 'rb'))
names.head()

,URL,Title,Visitors,Delta,Shift,ID,Date,Locate,Source_of_info
0,vesti.ru/doc.html?id=3066395,В России изменяются правила доставки пациентов...,16069,1,0,22557879,2018-10-04,ru,vesti.ru
1,www.mk.ru/science/2018/10/03/ameba-ubila-chelo...,"Амеба убила человека, съев его мозг изнутри",13867,15,0,115746545,2018-10-04,ru,mk.ru
2,novayagazeta.ru/articles/2018/09/29/77993-sud-...,Суд патриархата. Правосудие Тывы максимально л...,12315,0,0,49053795,2018-10-04,ru,novayagazeta.ru
3,www.mk.ru/politics/2018/10/01/v-rossii-gryadet...,В России грядет большая политическая буря,12298,1,0,114456809,2018-10-04,ru,mk.ru
4,www.mk.ru/moscow/2018/10/01/v-moskve-dvoe-poli...,В Москве двое полицейских расстреляли прохожег...,11053,0,0,113933922,2018-10-04,ru,mk.ru


In [9]:
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336000 entries, 0 to 335999
Data columns (total 9 columns):
URL               336000 non-null object
Title             336000 non-null object
Visitors          336000 non-null int64
Delta             336000 non-null int64
Shift             336000 non-null int64
ID                336000 non-null int64
Date              336000 non-null datetime64[ns]
Locate            336000 non-null object
Source_of_info    336000 non-null object
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 23.1+ MB


In [11]:
theme = theme_loader.Theme_loader('сбербанк', names) # ищем среди заголовков ключевое слово Сбербанк
theme.num() # количество статей с ключевым словом

27

In [12]:
theme.data_articles() # загружаем статьи в список, каждое препдложение отдельной строкой

100%|██████████| 27/27 [02:33<00:00,  5.58s/it]


In [15]:
data = pickle.load(open([name for name in os.listdir('./') if name.endswith('.pcl')][0], 'rb'))
data[0] # первая статья

['Если вы хотите поделиться интересным событием,                    воспользуйтесь данной формой',
 'На этот раз заседание назначено на 12 октября.',
 'В середине августа в Нижнекамске прошло первое заседание, на котором рассмотрели дело Сергея Коновалова, подозреваемого в краже более 15 млн рублей у ПАО «Сбербанк». Следствие считает, что подсудимый открывал валютный счет в банке, зная об ошибке мобильного приложения, которое при закрытии возвращало на карту двойную сумму денег.',
 'В результате ущерб от его деяний составил 15,4 млн рублей, которые принадлежали кредитной организации. Коновалов во время суда объяснил, что деньги банк переводил ему в виде займа по овердрафту. При этом мужчина не согласен отдавать всю сумму, так как с его счетов средства и списывались.',
 'Заседания несколько раз переносили по различным причинам. На этот раз прения сторон назначены на 12 октября.',
 'Если вы хотите поделиться интересным событием,    воспользуйтесь данной формой',
 'Президент Татарстана Ру